# Reinforcement Learning 2022-23: Assignment 3

**Deadline**: Mon 13 March 2023, 23:59


## Taxi Problem

In this assignment, you will solve the taxi problem for the following map:

````
        +---------+
        |R: | : :G|
        | : | : : |
        | : : : : |
        | | : | : |
        |Y| : |B: |
        +---------+
````

There are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends. The rewards are:

- -1 per step unless other reward is triggered.
- +20 delivering passenger.
- -10 executing "pickup" and "drop-off" actions illegally.

If a navigation action would cause the taxi to hit a wall (solid borders in the map), the action is a no-op, and there is only the usual reward of −1.

In [7]:
import numpy as np
from collections import defaultdict
import random
from tqdm import tqdm

Let us import the ``TaxiEnv`` class from env_taxi.py and create an instance of the Taxi environment.

In [8]:
from env_taxi import TaxiEnv
env=TaxiEnv()

Then, we reset the environment:

In [9]:
np.random.seed(100)
env.reset()

(2, 3, 2, 1)

It returns four state variables:

- Row index of the taxi (starting from 0 in Python)
- Colum index of the taxi (starting from 0 in Python)
- Passenger location (0-4): 0=R, 1=G, 2=Y, 3=B, 4=in taxi
- Destination location (0-3): same encoding rule as that for passenger location but excluding the "in taxi" option.

You can use the ``describe_state`` method of the Taxi instance to display the state variables without location encoding.

In [10]:
env.describe_state()

{'Taxi Location': [2, 3], 'Passenger Location': 'Y', 'Destination Index': 'G'}

You can use the ``render`` method to visualize the state.

In [11]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



There are 6 discrete deterministic actions:
- 'South': move south
- 'West': move north
- 'East': move east
- 'West': move west
- 'Pickup': pickup passenger
- 'Dropoff': drop off passenger

In [12]:
print(env.action_space)

['South', 'North', 'East', 'West', 'Pickup', 'Dropoff']


Let us move one step to west:

In [13]:
env.step('West')

((2, 2, 2, 1), -1, False)

The output is a 3-tuple: the new state (a list of 4 numbers), reward, and whether the episode is ended. 

Let us visualize the new state. Note that the last action is shown at the bottom as well.

In [14]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


**Exercise 1** Complete the codes below to generate _one_ episode for the taxi driver who:
- Picks up the passenger when the taxi is at the location of the passenger when they are not yet at the destination;
- Drops off the passenger in the taxi when reaching the destination;
- Moves randomly with equal probabilities when finding the passenger or when the passenger is in the taxi (but not yet at destination). 

Then print the sum of rewards for the taxi driver in your episode.

In [15]:
state=env.reset(271) # The number 271 fixes the initial state. Do not change it.
env.render()
print(env.describe_state())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

{'Taxi Location': [2, 3], 'Passenger Location': 'Y', 'Destination Index': 'B'}


Let us program the policy function for the taxi driver:

In [16]:
def naive_policy(state):
    ## Start Coding Here
    if (state[2]==4):
        if np.array_equal(env.locs[state[3]], state[0:2]):
            action = 'Dropoff'
        else:
            action = random.choice(['South', 'West', 'North', 'East'])
    else:   
        if np.array_equal(env.locs[state[2]], state[0:2]):
            action = 'Pickup'
        else:
            action = random.choice(['South', 'West', 'North', 'East'])
    ## End Coding Here
    
    return action

In [17]:
# Import the simulate_episode function from env_taxi.py to help you simulate episode(s)
from env_taxi import simulate_episode
help(simulate_episode)

Help on function simulate_episode in module env_taxi:

simulate_episode(env, policy, max_iter=inf)
    Simulate a episode following a given policy
    @param env: game environment
    @param policy: policy (a dictionary or a function)
    @return: resulting states, actions and rewards for the entire episode



In [18]:
# Set the seed to make results reproducible
np.random.seed(GroupNumber)

states, actions, rewards = simulate_episode(env,naive_policy)

# Save the sum of rewards in this variable G
G=np.sum(rewards)


# Print the return for this episode
print("Return for this episode:", G)

Return for this episode: -707


**Exercise 2** Following the steps below to investigate whether we can solve the optimal policy by using Monte Carlo Exploring Starts. If possible, complete the codes and save the optimal (greedy) policy as a dictionary/defaultdict object. Otherwise, comment on the issues you encounter. 

Follow the instructions below to complete the exercise.

In [19]:
# We import the simulate_episode_ES function from env_taxi.py to simulate episodes with exploring starts
from env_taxi import simulate_episode_ES
help(simulate_episode_ES)


# Let us try to simulate one episode with exploring start for the naive policy from Exercise 1
# Print the states, actions and rewards for an episode (the first 200 steps only)
np.random.seed(GroupNumber)
states, actions, rewards=simulate_episode_ES(env, naive_policy, 200)
print(states, actions, rewards)

Help on function simulate_episode_ES in module env_taxi:

simulate_episode_ES(env, policy, max_iter=inf)
    Simulate a episode with exploring starts
    @param env: game environment
    @param policy: policy (a dictionary or a function)
    @param max_iter: maximum number of time steps
    @return: resulting states, actions and rewards for the entire episode

[(2, 4, 2, 3), (2, 4, 2, 3), (2, 4, 2, 3), (3, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (4, 3, 2, 3), (4, 3, 2, 3), (4, 3, 2, 3), (3, 3, 2, 3), (3, 3, 2, 3), (3, 4, 2, 3), (3, 3, 2, 3), (4, 3, 2, 3), (4, 3, 2, 3), (3, 3, 2, 3), (4, 3, 2, 3), (4, 3, 2, 3), (4, 3, 2, 3), (4, 3, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (3, 4, 2, 3), (3, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (3, 4, 2, 3), (3, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (4, 4, 2, 3), (3, 4, 2, 3), (2, 4, 2, 3), (3, 4, 2, 3), (2, 4, 2, 3), (2, 4, 2, 3), (2, 4, 2, 3), (2, 4, 2, 3), (3, 4, 2, 3), (3, 4,

If you can solve the optimal policy by using Monte Carlo Exploring Starts, write down your codes in the next cell and save the optimal (greedy) policy in the dictionary object ``pi_ES`` initialized below. Use no more than 50000 episodes. We will import the ``performance_evaluation`` function from env_taxi.py to evaluate your policy in new episodes (that terminate within 1000 time steps).

In [20]:
pi_ES= defaultdict(lambda: np.random.choice(env.action_space))
states_t, actions_t, rewards_t = simulate_episode_ES(env, pi_ES, 200)
len(states_t[::-1][1:])

200

In [28]:
# Save your greedy actions as the values of this dictionary, whose keys are the states
pi_ES= defaultdict(lambda: np.random.choice(env.action_space))
np.random.seed(GroupNumber)

## Start Coding Here 
n_episode = 5000
G = defaultdict(float)

#initialization
Q = defaultdict(lambda: dict(zip(env.action_space, np.zeros(len(env.action_space)))))

for episode in tqdm(range(n_episode)):
    
    states_t, actions_t, rewards_t = simulate_episode_ES(env, pi_ES)
    
    # Calculate the returns for all state-action pairs in this episode
    return_t = {}
    G = 0
    for state_t, action_t, reward_t in zip(states_t[::-1][1:], actions_t[::-1], rewards_t[::-1]):
        G = gamma * g * reward_t
        if not(state_t)
        return_t[(state_t, action_t)] = G
        
     # Update the action-value estimate and policy
        for state_action, return_t in G.items():
    
    


## End Coding Here



# We import the performance_evaluation function from env_taxi.py to evaluate your optimal policy
from env_taxi import performance_evaluation
performance_evaluation(env,pi_ES)

SyntaxError: invalid syntax (2711260697.py, line 21)

If you cannot solve the optimal policy by using Monte Carlo Exploring Starts, explain the issue(s) and why they cannot be addressed.

Answer: The Monte Carlo method is not applicable to this problem in such setting since certain policies enter infinite loops disabling us to check a final reward for an episode as it does not terminate. Unlike Blackjack, Taxi problem can have policies leading an infinite long episode and a simple example of that can be choosing action 'West' at each state. Therefore, we are not able to start MC with Exploring Starts with such variety of available policies.

**Exercise 3**:  Find the optimal policy by using Q-learning with 50000 episodes and the exploration probability $\varepsilon=0.1$. Try two different values of the step-size parameter $\alpha=0.4$ and $\alpha=0.1$. Compare their performance in 10000 new episodes and comment on the similarities or/and differences. 

Follow the instructions below to complete the exercise.

In [22]:
def gen_epsilon_greedy_policy(action_space, Q, epsilon):
    """
    Generate epsilon greedy policy as a function
    @param action_space: list of all actions
    @param Q: the action values, Q[state] is a dictitonary
    @param epsilon: value of epsilon
    @return: epsilon-greedy policy as a function
    """
    def epsilon_greedy_policy(state):
        greedy_action=max(Q[state],key=Q[state].get)
        random_action=np.random.choice(action_space)
        # Starting Coding Here
        return np.random.choice([random_action,greedy_action],p=[epsilon,1-epsilon])
        # End Coding Here
    
    return epsilon_greedy_policy

In [23]:
np.random.seed(GroupNumber)

## Start Coding Here: Find the optimal policy by using Q-learning with 50000 episodes

def q_learning(env, gamma, n_episode, alpha, epsilon=0.1):
    """
    Off-policy Q-learning
    @param env: game environment
    @param gamma: discount factor
    @param n_episode: number of episodes
    @param alpha: step-size parameter
    @param epsilon: exploring probability
    @return: the optimal Q-function, the optimal policy, length of episode and the sum of rewards per episode
    """   
    Q = defaultdict(lambda: dict(zip(env.action_space, np.zeros(len(env.action_space)))))
    epsilon_greedy_policy = gen_epsilon_greedy_policy(env.action_space, Q, epsilon)
    
    pi= defaultdict(lambda: np.random.choice(env.action_space))
    
    for episode in tqdm(range(n_episode)):
        state = env.reset()
        is_done = False
        # Generate behavior using epsilon greedy policy
        while not is_done:
            action = epsilon_greedy_policy(state)
            next_state, reward, is_done = env.step(action)
            
            v= max(Q[next_state].values())
            td_delta = reward + gamma * v - Q[state][action]
            Q[state][action] += alpha * td_delta
            
            # Update Target Policy
            pi[state]=max(Q[state],key=Q[state].get)
            
            if is_done:
                break
            state = next_state
    
    return Q, pi
            
n_episode = 50000
epsilon = 0.1
gamma = 1

# Save the policy for alpha=0.4 as a Python dictionary/defaultdic object pi_qlearning_1
alpha = 0.4
Q_qlearning_1, pi_qlearning_1 = q_learning(env, gamma, n_episode, alpha, epsilon)

# Save the policy for alpha=0.1 as a Python dictionary/defaultdic object pi_qlearning_2
alpha = 0.1
Q_qlearning_2, pi_qlearning_2 = q_learning(env, gamma, n_episode, alpha, epsilon)



## End Coding Here

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [01:08<00:00, 734.28it/s]


In [29]:
print(Q_qlearning_1)

defaultdict(<function q_learning.<locals>.<lambda> at 0x00000185D9040A60>, {(2, 4, 2, 3): {'South': 3.908943437647326, 'North': 3.3949617553783855, 'East': 4.360707646266731, 'West': 5.999999999999969, 'Pickup': -4.3657830400000295, 'Dropoff': -4.178910032751499}, (3, 4, 2, 3): {'South': 1.3773622712870617, 'North': 4.336753121059448, 'East': 3.3699151200090283, 'West': 4.999999999999968, 'Pickup': -5.698945274579736, 'Dropoff': -6.249793493089751}, (4, 4, 2, 3): {'South': 2.6460162390621047, 'North': 3.9999999999999676, 'East': -9.662908653541294, 'West': -0.8792952430193437, 'Pickup': -8.736000000000015, 'Dropoff': -14.288826747127745}, (1, 4, 2, 3): {'South': 4.999999999999968, 'North': 0.32357412326435475, 'East': -0.7407107686400218, 'West': -3.4685170547107838, 'Pickup': -7.520045684095564, 'Dropoff': -8.361467514107579}, (2, 3, 2, 3): {'South': 4.999508148207604, 'North': 4.999890113610775, 'East': 4.999994411492221, 'West': 6.99999999999997, 'Pickup': -3.000042645491087, 'Dropo

In [24]:
# Compare their performance in 10000 new episodes: Do you find a big difference?
from env_taxi import performance_evaluation

print('--------  Q-learning, alpha=0.4  --------')
performance_evaluation(env,pi_qlearning_1)

print('--------  Q-learning, alpha=0.1  --------')
performance_evaluation(env,pi_qlearning_2)

--------  Q-learning, alpha=0.4  --------


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2186.01it/s]


The sum of rewards per episode: 7.8771
The percentage of episodes that cannot terminate within 1000 steps: 0.0
--------  Q-learning, alpha=0.1  --------


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2145.54it/s]

The sum of rewards per episode: 7.8886
The percentage of episodes that cannot terminate within 1000 steps: 0.0


Commment on the similarities or/and differences.

Answer: The difference between the average total rewards is relatively small. The result is just slightly better for the case of aplha=0.1. It may seem that in the Taxi Problem the scope of state-action pairs is limited to such extent that the optimal policy can be found in a comparable number of episodes for different values of alpha.

**Exercise 4**: Follow the steps below to find the optimal $\varepsilon$-soft policy by using Sarsa with 50000 episodes and the exploration probability $\varepsilon=0.1$. Try two different values of the step-size parameter $\alpha=0.4$ and $\alpha=0.1$. Compare their performance, and comment on the similarties or/and differences with that of Q-learning.

Follow the instructions below to complete the exercise.

In [25]:
np.random.seed(GroupNumber)

## Start Coding Here
def sarsa(env, gamma, n_episode, alpha, epsilon = 0.1):
    """
    Obtain the optimal policy with on-policy SARSA algorithm
    @param env: game environment
    @param gamma: discount factor
    @param n_episode: number of episodes
    @param alpha: step-size parameter
    @param epsilon: exploring probability
    @@return: the optimal Q-function, the optimal epsilon-greedy policy (as a function), length of episode and the sum of rewards per episode
    """    
    Q = defaultdict(lambda: dict(zip(env.action_space, np.zeros(len(env.action_space)))))   
    pi = gen_epsilon_greedy_policy(env.action_space, Q, epsilon)
    
    for episode in tqdm(range(n_episode)):
        state = env.reset()
        is_done = False
        action = pi(state)
        while not is_done:
            next_state, reward, is_done = env.step(action)
            next_action = pi(next_state)
            td_delta = reward + gamma * Q[next_state][next_action] - Q[state][action]
            Q[state][action] += alpha * td_delta

            if is_done:
                break
            state = next_state
            action = next_action
        
    return Q, pi

n_episode = 50000
epsilon = 0.1
gamma = 1

# Save the policy for alpha=0.4 in a Python function pi_sarsa_1
alpha = 0.4
Q_sarsa_1, pi_sarsa_1 = sarsa(env, gamma, n_episode, alpha, epsilon)
# Save the policy for alpha=0.1 in a Python function pi_sarsa_2
alpha = 0.1
Q_sarsa_2, pi_sarsa_2 = sarsa(env, gamma, n_episode, alpha, epsilon)

# End Coding Here



100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [01:10<00:00, 713.51it/s]


In [26]:
from env_taxi import performance_evaluation

print('--------  Sarsa, alpha=0.4  --------')
performance_evaluation(env,pi_sarsa_1)

print('--------  Sarsa, alpha=0.1  --------')
performance_evaluation(env,pi_sarsa_2)

--------  Sarsa, alpha=0.4  --------


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:22<00:00, 447.15it/s]


The sum of rewards per episode: -15.0227
The percentage of episodes that cannot terminate within 1000 steps: 0.0
--------  Sarsa, alpha=0.1  --------


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 855.03it/s]

The sum of rewards per episode: 2.3437
The percentage of episodes that cannot terminate within 1000 steps: 0.0


Compare their performance, and comment on the similarties or/and differences with that of Q-learning.

Answer: In case of Sarsa we can notice a significant difference in performance between different alphas with alpha=0.1 being better. For alpha=0.4, Sarsa seemingly adjusts the state-action values too abruptly leading to non-optimal policies which gives such a bad result especially considering that we look only at the first 50000 episodes. These Sarsa outcomes are much worse than both Q-learning results.

**Exercise 5**: Do you expect double Q-learning will improve Q-learning substantially? Motivate your answer. You do not need to implement the double Q-learning algorithm.

Answer: We would not expect a considerable change. The state-action space is rather limited. Hence, there are no significant options for some state-action pairs to give similar Q-values and hence misleading the algorithm in the search for the optimal policy.

Furthermore, looking at the Q-values, they are not overestimated relative to the rewards. Double Q-learning is much better than regular Q-learning when these values are overestiamted. 

**Exercise 6** 
Follow the steps below to investigate whether we can solve the optimal policy by using off-policy Monte Carlo control with weighted importance sampling. Use the optimal $\varepsilon$-soft policy found by Sarsa above with $\alpha=0.1$ as the behavior policy and use no more than 50000 episodes.

In [27]:
np.random.seed(GroupNumber)

## Start Coding Here
def mc_control_off_policy_WIS(env, b, Q_b, gamma, n_episode):
    """
    Obtain the optimal policy using on-policy MC control method with exploring starts
    @param env: game environment
    @param gamma: discount factor
    @param n_episode: number of episodes
    @return: the optimal Q-function, and the optimal policy (as a dictionary)
    
    """
    Q = defaultdict(lambda: dict(zip(env.action_space, np.zeros(len(env.action_space))))) 
    C = defaultdict(lambda: dict(zip(env.action_space, np.zeros(len(env.action_space))))) 
    pi = defaultdict(lambda: np.random.choice(env.action_space))
    
    for episode in tqdm(range(n_episode)):
        states_t, actions_t, rewards_t = simulate_episode(env,b)
        G = 0
        W = 1
        for state_t, action_t, reward_t in zip(states_t[::-1][1:], actions_t[::-1], rewards_t[::-1]):
            G = gamma * G + reward_t
            C[(state_t, action_t)] = C[(state_t, action_t)] + W
            Q[(state_t, action_t)] = Q[(state_t, action_t)] + W/C[(state_t, action_t)]*(G-Q[(state_t, action_t)])
            
            # Update Target Policy
            pi[state_t]=max(Q[state_t],key=Q[state_t].get)
            
            if not pi[state_t] == action_t:
                break
            
            b_AS
            if pi_opt(states[t]) == actions[t]
            W = W/(1-(len(env.action_space)-1)*epsilon/len(env.action_space)) if pi_opt(states[t]) == actions[t] else W/(epsilon/6)
    
    
    
    
    

# Save your optimal policy as a dictionary/defaultdict Python object pi_wis






## End Coding Here

SyntaxError: invalid syntax (1182042131.py, line 32)

In [ ]:
x = np.array([[1,2,3],[4,5,6]])
x[::-1][1:]
states_t, actions_t, rewards_t = simulate_episode(env,naive_policy)
print(len(states_t),len(actions_t))

Check the total number of distinct states visited in your episodes. Have you reached all 400 possible states (within the episode)?

In [ ]:
print('Number of States Visited:', len(pi_wis))

Generate a complete policy that:
- follows your optimal policy (in this exercise) if the state is visited in your episodes (in this exercise);
- otherwise follows the naive policy in Exercise 1.

If you have reached all possible states in your episodes, it is identical to your optimal policy.

In [ ]:
def complete_policy(state):
    ## Start Coding Here
    
    
    
    
    ## End Coding Here
    return action

Let us evaluate its performance:

In [ ]:
performance_evaluation(env,complete_policy)

**Exercise 7**: What are your conclusions from the analysis above? What are the advantages and disadvantages of these methods?

Answer: